In [ ]:
# claude - visualizer - saving - pipeline bulldog
import open3d as o3d
import numpy as np

DEFAULT_PARAMS = {
    'body_scale': [1.8, 1.2, 0.7],  # Wider chest, shorter height
    'head_scale': [0.9, 0.7, 0.6],  # Squarer head shape
    'leg_radius': 0.2,              # Thicker legs
    'jaw_depth': 0.4,               # Bulldog jaw
    'wrinkle_density': 0.1,         # Density of facial wrinkles
    'chest_depth': 1.4,             # Pronounced chest
    'shoulder_width': 1.6           # Wide shoulders
}

def create_wrinkled_mesh(base_mesh, wrinkle_density):
    """Add wrinkle deformation to mesh vertices"""
    vertices = np.asarray(base_mesh.vertices)
    # Add random displacement to simulate wrinkles
    noise = np.random.normal(0, wrinkle_density, vertices.shape)
    vertices += noise
    base_mesh.vertices = o3d.utility.Vector3dVector(vertices)
    return base_mesh

class EnhancedBulldogGenerator:
    def __init__(self):
        self.components = []
        self.params = DEFAULT_PARAMS
        
    def _create_head(self):
        """Complex bulldog head with wrinkles and proper jaw structure"""
        # Main head sphere for rounded shape
        head = create_part('sphere', {'radius': 1})
        self._apply_scaling(head, self.params['head_scale'])
        
        # Create muzzle using truncated cone
        muzzle = create_part('cone', {
            'radius': 0.6,
            'height': 0.4
        })
        muzzle.translate((self.params['body_scale'][0]/2 + 0.3, 0, 0.3))
        
        # Add wrinkles to head
        head = create_wrinkled_mesh(head, self.params['wrinkle_density'])
        
        # Create jowls using small spheres
        left_jowl = create_part('sphere', {'radius': 0.3})
        right_jowl = create_part('sphere', {'radius': 0.3})
        left_jowl.translate((self.params['body_scale'][0]/2, 0.4, 0.2))
        right_jowl.translate((self.params['body_scale'][0]/2, -0.4, 0.2))
        
        self.components.extend([head, muzzle, left_jowl, right_jowl])

    def _create_body(self):
        """Muscular bulldog body with proper chest structure"""
        # Main body
        body = create_part('sphere', {'radius': 1})
        self._apply_scaling(body, [
            self.params['body_scale'][0],
            self.params['body_scale'][1],
            self.params['chest_depth']
        ])
        
        # Add pronounced shoulders
        left_shoulder = create_part('sphere', {'radius': 0.4})
        right_shoulder = create_part('sphere', {'radius': 0.4})
        left_shoulder.translate((0.2, self.params['shoulder_width']/2, 0))
        right_shoulder.translate((0.2, -self.params['shoulder_width']/2, 0))
        
        self.components.extend([body, left_shoulder, right_shoulder])

    def _create_legs(self):
        """Articulated legs with proper joint structure"""
        for x, y in [(0.6, 0.6), (-0.6, 0.6), (0.6, -0.6), (-0.6, -0.6)]:
            # Upper leg
            upper = create_part('cylinder', {
                'radius': self.params['leg_radius'],
                'height': self.params['body_scale'][2] * 0.4
            })
            upper.translate((x, y, -self.params['body_scale'][2]/3))
            
            # Lower leg
            lower = create_part('cylinder', {
                'radius': self.params['leg_radius'] * 0.8,
                'height': self.params['body_scale'][2] * 0.4
            })
            lower.translate((x, y, -self.params['body_scale'][2]))
            
            # Paw
            paw = create_part('sphere', {'radius': self.params['leg_radius'] * 1.2})
            paw.translate((x, y, -self.params['body_scale'][2] * 1.2))
            
            self.components.extend([upper, lower, paw])

    def generate(self):
        self._create_body()
        self._create_head()
        self._create_legs()
        
        # Merge all components
        bulldog = self.components[0]
        for part in self.components[1:]:
            bulldog += part
        
        return bulldog

In [ ]:
# builds but sux
import open3d as o3d
import numpy as np

def create_part(shape, params, position=(0,0,0), rotation=None):
    """Create a geometric part with transformation"""
    mesh = getattr(o3d.geometry.TriangleMesh, f'create_{shape}')(**params)
    mesh.compute_vertex_normals()
    
    # Create transformation matrix
    transform = np.eye(4)
    transform[:3, 3] = position  # Set translation
    
    if rotation:
        # Convert rotation angles to radians and create rotation matrix
        theta_x, theta_y, theta_z = np.radians(rotation)
        Rx = np.array([[1, 0, 0],
                      [0, np.cos(theta_x), -np.sin(theta_x)],
                      [0, np.sin(theta_x), np.cos(theta_x)]])
        
        Ry = np.array([[np.cos(theta_y), 0, np.sin(theta_y)],
                      [0, 1, 0],
                      [-np.sin(theta_y), 0, np.cos(theta_y)]])
        
        Rz = np.array([[np.cos(theta_z), -np.sin(theta_z), 0],
                      [np.sin(theta_z), np.cos(theta_z), 0],
                      [0, 0, 1]])
        
        R = Rz @ Ry @ Rx
        transform[:3, :3] = R
    
    mesh.transform(transform)
    return mesh

DEFAULT_PARAMS = {
    'body_scale': [1.8, 1.2, 0.7],  # Wider chest, shorter height
    'head_scale': [0.9, 0.7, 0.6],  # Squarer head shape
    'leg_radius': 0.2,              # Thicker legs
    'jaw_depth': 0.4,               # Bulldog jaw
    'wrinkle_density': 0.1,         # Density of facial wrinkles
    'chest_depth': 1.4,             # Pronounced chest
    'shoulder_width': 1.6           # Wide shoulders
}

def create_wrinkled_mesh(base_mesh, wrinkle_density):
    """Add wrinkle deformation to mesh vertices"""
    vertices = np.asarray(base_mesh.vertices)
    # Add random displacement to simulate wrinkles
    noise = np.random.normal(0, wrinkle_density, vertices.shape)
    vertices += noise
    base_mesh.vertices = o3d.utility.Vector3dVector(vertices)
    return base_mesh

class EnhancedBulldogGenerator:
    def __init__(self):
        self.components = []
        self.params = DEFAULT_PARAMS
    
    def _apply_scaling(self, mesh, scale_factors):
        """Proper non-uniform scaling using transformation matrix"""
        scaling_matrix = np.eye(4)
        scaling_matrix[0, 0] = scale_factors[0]  # X-axis
        scaling_matrix[1, 1] = scale_factors[1]  # Y-axis
        scaling_matrix[2, 2] = scale_factors[2]  # Z-axis
        mesh.transform(scaling_matrix)
        return mesh
        
    def _create_head(self):
        """Complex bulldog head with wrinkles and proper jaw structure"""
        # Main head sphere for rounded shape
        head = create_part('sphere', {'radius': 1})
        self._apply_scaling(head, self.params['head_scale'])
        
        # Create muzzle using truncated cone
        muzzle = create_part('cone', {
            'radius': 0.6,
            'height': 0.4
        })
        muzzle.translate((self.params['body_scale'][0]/2 + 0.3, 0, 0.3))
        
        # Add wrinkles to head
        head = create_wrinkled_mesh(head, self.params['wrinkle_density'])
        
        # Create jowls using small spheres
        left_jowl = create_part('sphere', {'radius': 0.3})
        right_jowl = create_part('sphere', {'radius': 0.3})
        left_jowl.translate((self.params['body_scale'][0]/2, 0.4, 0.2))
        right_jowl.translate((self.params['body_scale'][0]/2, -0.4, 0.2))
        
        self.components.extend([head, muzzle, left_jowl, right_jowl])

    def _create_body(self):
        """Muscular bulldog body with proper chest structure"""
        # Main body
        body = create_part('sphere', {'radius': 1})
        self._apply_scaling(body, [
            self.params['body_scale'][0],
            self.params['body_scale'][1],
            self.params['chest_depth']
        ])
        
        # Add pronounced shoulders
        left_shoulder = create_part('sphere', {'radius': 0.4})
        right_shoulder = create_part('sphere', {'radius': 0.4})
        left_shoulder.translate((0.2, self.params['shoulder_width']/2, 0))
        right_shoulder.translate((0.2, -self.params['shoulder_width']/2, 0))
        
        self.components.extend([body, left_shoulder, right_shoulder])

    def _create_legs(self):
        """Articulated legs with proper joint structure"""
        for x, y in [(0.6, 0.6), (-0.6, 0.6), (0.6, -0.6), (-0.6, -0.6)]:
            # Upper leg
            upper = create_part('cylinder', {
                'radius': self.params['leg_radius'],
                'height': self.params['body_scale'][2] * 0.4
            })
            upper.translate((x, y, -self.params['body_scale'][2]/3))
            
            # Lower leg
            lower = create_part('cylinder', {
                'radius': self.params['leg_radius'] * 0.8,
                'height': self.params['body_scale'][2] * 0.4
            })
            lower.translate((x, y, -self.params['body_scale'][2]))
            
            # Paw
            paw = create_part('sphere', {'radius': self.params['leg_radius'] * 1.2})
            paw.translate((x, y, -self.params['body_scale'][2] * 1.2))
            
            self.components.extend([upper, lower, paw])

    def generate(self):
        self._create_body()
        self._create_head()
        self._create_legs()
        
        # Merge all components
        bulldog = self.components[0]
        for part in self.components[1:]:
            bulldog += part
        
        return bulldog

# Generate and visualize the bulldog
generator = EnhancedBulldogGenerator()
bulldog = generator.generate()

# Print component types
print("Bulldog components:", [type(m).__name__ for m in generator.components])

# Save the mesh to the root directory
o3d.io.write_triangle_mesh("./bulldog.ply", bulldog)

# Visualize
o3d.visualization.draw_geometries([bulldog],
                                window_name="Enhanced Bulldog",
                                width=800,
                                height=600)

Bulldog components: ['TriangleMesh', 'TriangleMesh', 'TriangleMesh', 'TriangleMesh', 'TriangleMesh', 'TriangleMesh', 'TriangleMesh', 'TriangleMesh', 'TriangleMesh', 'TriangleMesh', 'TriangleMesh', 'TriangleMesh', 'TriangleMesh', 'TriangleMesh', 'TriangleMesh', 'TriangleMesh', 'TriangleMesh', 'TriangleMesh', 'TriangleMesh']
